# Real Life Atmospheric Data!

Firstly, cast this magic spell to get some string paths to data files.

In [ ]:
#!git clone https://github.com/met-office-lab/asn_data_utils.git
from asn_data_utils.asn_utils.Loader import Loader
l = Loader()
fs = l.list_files("mogreps")

Now we can make two sets of files.

This first one gives us a list of file paths on the same node as this Notebook i.e. local loading. We're just doing this for one day.

In [58]:
local_fs_for_20161002T0000Z = [f for f in fs if '20161002T0000Z' in f and '_000_' not in f]

And this second one gives us the same files but now for thier paths on the Dask nodes. All files and can be accessed from any node.

In [61]:
node_fs_for_20161002T0000Z = [f.replace('/usr/local/share/notebooks/', '/') for f in local_fs_for_20161002T0000Z]

We're going to use Iris, the Met Office's Python data analysis module. Here's it's going to load a bunch of the data files locally into separate data objects, called `cubes`

In [62]:
import iris
ds = iris.load_raw(fs_for_20161002T0000Z[:30], "precipitation_amount")

/opt/conda/lib/python3.5/site-packages/iris/fileformats/_ff.py:732: UserWarning: The STASH code m10s20i033 was not found in the STASH to grid type mapping. Picking the P position as the cell type
  'position as the cell type'.format(stash))
/opt/conda/lib/python3.5/site-packages/iris/fileformats/rules.py:80: UserWarning: Multiple reference cubes for orography
  .format(self.name))


In [63]:
print(ds)

0: precipitation_amount / (kg m-2)     (latitude: 600; longitude: 800)
1: precipitation_amount / (kg m-2)     (latitude: 600; longitude: 800)
2: precipitation_amount / (kg m-2)     (latitude: 600; longitude: 800)
3: precipitation_amount / (kg m-2)     (latitude: 600; longitude: 800)
4: precipitation_amount / (kg m-2)     (latitude: 600; longitude: 800)
5: precipitation_amount / (kg m-2)     (latitude: 600; longitude: 800)
6: precipitation_amount / (kg m-2)     (latitude: 600; longitude: 800)
7: precipitation_amount / (kg m-2)     (latitude: 600; longitude: 800)
8: precipitation_amount / (kg m-2)     (latitude: 600; longitude: 800)
9: precipitation_amount / (kg m-2)     (latitude: 600; longitude: 800)
10: precipitation_amount / (kg m-2)     (latitude: 600; longitude: 800)
11: precipitation_amount / (kg m-2)     (latitude: 600; longitude: 800)
12: precipitation_amount / (kg m-2)     (latitude: 600; longitude: 800)
13: precipitation_amount / (kg m-2)     (latitude: 600; longitude: 800)
14

Each cube is a numpy array (`ds[0].data`) with associated metadata. Using the magic of Iris we can turn these into one cube.

In [65]:
d = ds.merge_cube()
print(d)

precipitation_amount / (kg m-2)     (-- : 90; latitude: 600; longitude: 800)
     Dimension coordinates:
          latitude                      -             x               -
          longitude                     -             -               x
     Auxiliary coordinates:
          forecast_period               x             -               -
          realization                   x             -               -
          time                          x             -               -
     Scalar coordinates:
          forecast_reference_time: 2016-10-02 00:00:00
     Attributes:
          STASH: m01s05i226
          source: Data from Met Office Unified Model
          um_version: 10.2
     Cell methods:
          mean: time (1 hour)


So this now looks like one cube, but its actually made up of lots of separate cubes from separate files...

...so can we put these files on different nodes?

### Your challenge, should you choose to accept it...
Is to see if you can do distributed processing on this data. We've got another 1EB of it, so it would be rather lovely if we could crack this!